# pipeline

作業で終わってしまう。

In [2]:
from transformers import pipeline
transcriber = pipeline(task="automatic-speech-recognition")
transcriber("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebo

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


{'text': 'I HAVE A DREAM BUT ONE DAY THIS NATION WILL RISE UP LIVE UP THE TRUE MEANING OF ITS TREES'}

# Modelを使う


In [3]:
import soundfile as sf
import librosa
import IPython.display as ipd
import numpy as np

def readAudioFile(audio_path):
    audio, sample_rate = sf.read(audio_path)
    if audio.ndim > 1:
        audio = audio[:, 0]
    if sample_rate != 16000:
        audio = librosa.resample(audio, orig_sr=sample_rate, target_sr=16000)
    audio = audio.astype("float32")
    # 先頭に0.5秒の無音を追加
    audio = np.concatenate([np.zeros(int(0.5 * 16000)), audio])
    return audio

audio=readAudioFile("/mnt/e/Download/ja_test_0/common_voice_ja_19485250.mp3")
ipd.Audio(audio, rate=16000)

In [4]:
from transformers import  WhisperModel,WhisperTokenizer,WhisperFeatureExtractor
MODEL_NAME="openai/whisper-medium"
#MODEL_NAME="distil-whisper/distil-large-v2"
DEVICE="cuda:0"
feature_extractor=WhisperFeatureExtractor.from_pretrained(MODEL_NAME)
features = feature_extractor(audio, sampling_rate=16000, return_tensors="pt").to(DEVICE)
input_features = features.input_features
input_features.shape

torch.Size([1, 80, 3000])

In [5]:
from transformers import WhisperForConditionalGeneration,AutoProcessor
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
model.to(DEVICE)
processor = AutoProcessor.from_pretrained(MODEL_NAME)
model.config.forced_decoder_ids=processor.get_decoder_prompt_ids(
    task="transcribe", language="japanese",no_timestamps=True
)
model.config.forced_decoder_ids
processor.batch_decode(model.config.forced_decoder_ids)

['"<|ja|>', '#<|transcribe|>', '$<|notimestamps|>']

In [6]:
audio=readAudioFile("/mnt/e/dataset/jsut_ver1.1/basic5000/wav/BASIC5000_0130.wav")
features = feature_extractor(audio, sampling_rate=16000, return_tensors="pt").to(DEVICE)
input_features = features.input_features

output = model.generate(input_features,return_dict_in_generate=True,output_scores=True)
processor.batch_decode(output.sequences, skip_special_tokens=True)

['ほうぎきはますますはげしくなってきたので、わたしたちはしってちかしつにおりていったのよ。']

# Decoderに細工する


In [7]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(MODEL_NAME)
def get_tokens_as_tuple(word):
    return tuple(tokenizer([word], add_special_tokens=False).input_ids[0])

[get_tokens_as_tuple("砲撃"),get_tokens_as_tuple("祭司")]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[(23230, 110, 20559, 225), (12695, 255, 32981)]

In [33]:
audio=readAudioFile("/mnt/e/dataset/jsut_ver1.1/basic5000/wav/BASIC5000_0130.wav")
features = feature_extractor(audio, sampling_rate=16000, return_tensors="pt").to(DEVICE)
input_features = features.input_features

output = model.generate(input_features,return_dict_in_generate=True,output_scores=True,num_beams=4)
text = processor.batch_decode(output.sequences, skip_special_tokens=True)
text

LibsndfileError: Error opening '/mnt/e/dataset/jsut_ver1.1//wav/BASIC5000_0130.wav': System error.

In [26]:

output = model.generate(input_features,return_dict_in_generate=True,output_scores=True,num_beams=4)
text = processor.batch_decode(output.sequences, skip_special_tokens=True)
text

['足を水につけて汚れを落としなさい。']

In [21]:
def create_sequence_bias(tokens, bias):
    sequence_bias = {}
    for token in tokens:
        token_ids = get_tokens_as_tuple(token)
        # 先頭のトークンだけが入ったタプルを作成
        head_token = (token_ids[0],)
        sequence_bias[token_ids] = bias
        sequence_bias[head_token] = bias
    return sequence_bias


sequence_bias = create_sequence_bias(["祭司","砲撃","小切手"],1.0)
sequence_bias


{(12695, 255, 32981): 1.0,
 (12695,): 1.0,
 (23230, 110, 20559, 225): 1.0,
 (23230,): 1.0,
 (7322, 23632, 11389): 1.0,
 (7322,): 1.0}

In [32]:

output2 = model.generate(input_features,return_dict_in_generate=True,output_scores=True,sequence_bias=sequence_bias,num_beams=4)
biased_text=processor.batch_decode(output2.sequences, skip_special_tokens=True)

[
    {"text":text,"seq": output.sequences[0][4:]},
    {"text":biased_text,"seq":output2.sequences[0][4:]},
    sequence_bias
]

[{'text': ['私はドナルドソンさんの助手のものです。'],
  'seq': tensor([20083,  3065, 11195, 26719,  9405, 11195, 42668,  4824, 15567,  2972,
          37618, 11389,  2972, 44726,  4767,  1543, 50257], device='cuda:0')},
 {'text': ['私はドナルドソンさんの助手のものです。'],
  'seq': tensor([20083,  3065, 11195, 26719,  9405, 11195, 42668,  4824, 15567,  2972,
          37618, 11389,  2972, 44726,  4767,  1543, 50257], device='cuda:0')},
 {(12695, 255, 32981): 1.0,
  (12695,): 1.0,
  (23230, 110, 20559, 225): 1.0,
  (23230,): 1.0,
  (7322, 23632, 11389): 1.0,
  (7322,): 1.0}]